MODELO DE CRÉDITO PARA PRÉSTAMOS PERSONALES

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
"""
Función para cargar la información para entrenar y validar el modelo. 
La información no será proporcionada por el usuario final, sino que ya estará establecida. Se utilizará el archivo 
con los datos limpios y ordenados que se realizó para el modelo tradicional. 
"""
def get_data(data):
    return pd.read_csv(data)

In [3]:
"""
Función para separar la data en train y test. 
Se utilizará el 70% de los datos totales para entrenar el modelo y el 30% restante para verificar su funcionamiento. 
La división de los datos se realizará de manera aleatoria, establecionedo una semilla para poder replicar los 
resultados obtenidos. 

EXPLICAR TARGET COLUMN
"""
def split_train_test(data, target_column, random_state=100):
    data_train, data_test = train_test_split(
        data,
        test_size=0.3,
        random_state=random_state,
        stratify=data[target_column]  
    )
    return data_train.reset_index(drop=True), data_test.reset_index(drop=True)

In [4]:
"""
Función para preparar los datos. 
Ya que en los modelos que se utilizarán más adelante los datos deben de estar separados en X y Y, además de estar 
normalizados (unicamente los valores numéricos), se optó por crear una función que realizara esta preparación. Para utilizarla, es necesario alimentar a esta
función con los datos seleccionados para train y para test obtenidos en la función anterior. 
"""

def prepare_variables(data_train, data_test):
    X_train = data_train.drop('Credit_Score', axis=1)
    y_train = data_train['Credit_Score']
    X_test = data_test.drop('Credit_Score', axis=1)
    y_test = data_test['Credit_Score']

    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X_train.select_dtypes(exclude=['int64', 'float64']).columns

    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

    return X_train_scaled, X_test_scaled, y_train, y_test

In [5]:
def preprocess_data(X_train, X_test, y_train, y_test):
    X_train_processed = X_train.copy()
    X_test_processed = X_test.copy()
    
    X_train_processed = pd.get_dummies(X_train_processed, 
                                     columns=['Payment_of_Min_Amount'], 
                                     drop_first=True)
    
    X_test_processed = pd.get_dummies(X_test_processed, 
                                    columns=['Payment_of_Min_Amount'], 
                                    drop_first=True)
    
    missing_cols = set(X_train_processed.columns) - set(X_test_processed.columns)
    for col in missing_cols:
        X_test_processed[col] = 0
    X_test_processed = X_test_processed[X_train_processed.columns]
    
    ordinal_mapping = {'Bad': 0, 'Standard': 1, 'Good': 2}
    X_train_processed['Credit_Mix'] = X_train_processed['Credit_Mix'].map(ordinal_mapping)
    X_test_processed['Credit_Mix'] = X_test_processed['Credit_Mix'].map(ordinal_mapping)
    
    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)
    y_test_encoded = le.transform(y_test)
    
    return X_train_processed, X_test_processed, y_train_encoded, y_test_encoded

In [41]:
"""
Función para crear un primer modelo con el método de Random Forest. 
Esta función entrará con los datos normalizados para poder proponer una predicción y una probabilidad asociada. 

EXPLICAR UN POCO MÁS QUE ES RF

"""

#RF FALTA DOCUMENTACION Y CAMBIAR COLUMNA
def random_forest(X_train,X_test, y_train):
    rf_model = RandomForestClassifier(random_state=100)
    rf_model.fit(X_train, y_train)
    rf_probs = rf_model.predict_proba(X_test)[:, 1]
    rf_preds = rf_model.predict(X_test)
    
    return rf_probs,rf_preds

In [36]:
def logistic_regression(X_train, y_train, X_test):
    lr_model = LogisticRegression(max_iter=1000, random_state=100)
    lr_model.fit(X_train, y_train)
    lr_probs = lr_model.predict_proba(X_test)[:, 1]
    lr_preds = lr_model.predict(X_test)
    
    return lr_probs,lr_preds

In [58]:
def catboost_model(X_train, y_train, X_test=None, cat_features=None, return_proba=True, 
                  iterations=1000, learning_rate=0.03, depth=6, verbose=0, **params):
    
    model = CatBoostClassifier(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        cat_features=cat_features,
        verbose=verbose,
        **params
    )
    
    model.fit(X_train, y_train)
    
    if X_test is not None:
        if return_proba and hasattr(model, 'predict_proba'):
            probs = model.predict_proba(X_test)[:, 1]  # Probability of positive class
        else:
            probs = None
        preds = model.predict(X_test)
        return probs, preds
    
    return model

In [22]:
# Crear el modelo CatBoost
def catboost_model(X_train_scaled, y_train_scaled, cat_features=None, iterations=1000, learning_rate=0.03, depth=6):
    model = CatBoostClassifier(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        cat_features=cat_features,
        verbose=0 
    )
    
    return model.fit(X_train_scaled, y_train_scaled)

In [53]:
def xgboost_model(X_train,X_test,y_train, task_type='classification', **params):
   
    if task_type == 'classification':
        model = XGBClassifier(**params)
    else:
        model = XGBRegressor(**params)
        
    model.fit(X_train, y_train)
    xgrobs = model.predict_proba(X_test)[:, 1]
    xgreds = model.predict(X_test)
    
    return xgrobs,xgreds

In [24]:
def get_roc_metrics(y_test, probs):
    fpr, tpr, _ = roc_curve(y_test, probs)
    auc = roc_auc_score(y_test, probs)
    return fpr, tpr, auc

In [25]:
"""
Función para cargar la info del modelo
Separar en trian y test 
Función para entrenador el modelo con RF
Función para entrenador el modelo con Regresion 
Función para entrenador el modelo con ---
fUNCION PARA COMBINARLOS
Función para evaluar los modelos 
Fuincion Auroc
Funcion matriz
Función para que el usuario inserte el cliente a evaluar
¿Qué hacer en caso de que no se tenga un dato?
"""

'\nFunción para cargar la info del modelo\nSeparar en trian y test \nFunción para entrenador el modelo con RF\nFunción para entrenador el modelo con Regresion \nFunción para entrenador el modelo con ---\nfUNCION PARA COMBINARLOS\nFunción para evaluar los modelos \nFuincion Auroc\nFuncion matriz\nFunción para que el usuario inserte el cliente a evaluar\n¿Qué hacer en caso de que no se tenga un dato?\n'

In [26]:
"""
Función para cargar y leer los datos con la información crediticia que se utilizará para entrenar y validar el modelo.

El archivo que contenga estos datos debe de ser presentado en formato csv, de lo contrario, un mensaje de error hará saber
al usuario que el archivo no cumple con el formato solicitado.
El archivo debe de contener las siguientes 12 columnas en el orden que a continuación se presenta, cuidando
que estas correspondan a las primeras 12 columnas del documento: Interest_Rate, Delay_from_due_date,
Payment_of_Min_Amount, Outstanding_Debt, Num_Credit_Inquiries, Num_of_Delayed_Payment, Num_of_Loan, 
Credit_Utilization_Ratio, Num_Bank_Accounts, Num_Credit_Card, Credit_Mix, Credit_History_Age. 

Clasifiación para las variables que no son numéricas:
•Payment_of_Min_Amount: Yes, No, NM (Not Mentioned)
•Credit_Mix: Good, Standard, Poor

También es importante mencionar que el Interest_Rate se debe indicar como números enteros, no como porcentaje.
Para Outstanding_Debt, Credit_Utilization_Ratio y Credit_History_Age se pueden usar puntos decimales y para las 
restantes son números enteros.
El Delay_from_due_date debe indicarse en número de días.

En caso de tener valores faltantes se pondra cero para el caso de las columnas numéricas y NA para las categóricas. 
"""
def insert_credit_info(info):
    if not info.lower().endswith('.csv'):
        raise ValueError("El formato del archivo proporcionado no concuerda con el solicitado.")
    
    return pd.read_csv(info)

#Evaluar el modelo en lso datos porporcionado

In [27]:
data = "data_train_cleaned_final.csv"

In [28]:
data = get_data(data)

In [29]:
data_train, data_test = split_train_test(data,target_column="Credit_Score", random_state=100)

In [30]:
X_train_scaled,X_test_scaled, y_train, y_test= prepare_variables(data_train,data_test)

In [31]:
X_train_processed, X_test_processed, y_train_encoded, y_test_encoded= preprocess_data(X_train_scaled, X_test_scaled, y_train, y_test)

In [39]:
probs, preds = logistic_regression(
    X_train_processed, 
    y_train_encoded, 
    X_test_processed
)

array([0.12663436, 0.31477633, 0.07205469, ..., 0.21866146, 0.60497311,
       0.51697636])

In [43]:
rf_probs,rf_preds = random_forest(X_train_processed,X_test_processed, y_train_encoded)

In [56]:
xgb_params = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'random_state': 42
}
xgrobs,xgreds= xgboost_model(X_train_processed,X_test_processed, y_train_encoded, task_type='classification', **xgb_params)

In [60]:
probs, preds = catboost_model(X_train_scaled, y_train, X_test_scaled)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=2]="No": Cannot convert 'b'No'' to float

In [61]:
categorical_cols = X_train_scaled.select_dtypes(include=['object', 'category']).columns.tolist()
print("Columnas categóricas:", categorical_cols)

Columnas categóricas: ['Payment_of_Min_Amount', 'Credit_Mix']


In [62]:
def catboost_model(X_train, y_train, X_test=None, return_proba=True, **params):
    from catboost import CatBoostClassifier, Pool
    
    # Identificar automáticamente columnas categóricas
    cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Convertir a Pool para manejo óptimo
    train_pool = Pool(
        data=X_train,
        label=y_train,
        cat_features=cat_features
    )
    
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.03,
        depth=6,
        verbose=0,
        **params
    )
    
    model.fit(train_pool)
    
    if X_test is not None:
        test_pool = Pool(
            data=X_test,
            cat_features=cat_features
        )
        if return_proba:
            probs = model.predict_proba(test_pool)[:, 1]
        else:
            probs = None
        preds = model.predict(test_pool)
        return probs, preds
    
    return model

In [63]:
# Convertir columnas categóricas a tipo 'category'
for col in ['Payment_of_Min_Amount', 'Credit_Mix']:
    X_train_scaled[col] = X_train_scaled[col].astype('category')
    X_test_scaled[col] = X_test_scaled[col].astype('category')

In [64]:
probs, preds = catboost_model(
    X_train_scaled,
    y_train,
    X_test=X_test_scaled
)